# Atividade 01 - Algoritmo KNN

**IMD1101** Aprendizado de Máquina - 2023.2 IMD/UFRN

Aluno: João Guilherme Lopes Alves da Costa

Matrícula: 20200045609

Nesta atividade foi implementado o algoritmo k-NN (k-nearest neighbors) utilizando Python para treinamento de um modelo.

Inicialmente, importar todas as bibliotecas necessárias:

In [2]:
from sklearn import datasets
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter

A base de dados que será utilizada é a `iris` da biblioteca Sklearn. Neste caso não será necessário fazer o tratamento dos dados.

In [3]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

Para visualizarmos os dados que estamos trabalhando, utilizando a biblioteca Pandas:

In [4]:
df = pd.DataFrame(X, columns = iris.feature_names)
display(df)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


Agora, para poder utilizar o algoritmo KNN, devemos dividir o conjunto de dados de treinamento e teste. Neste caso o conjunto de teste terá 20% do dados:

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
print(X_train.shape)
print(X_test.shape)

(120, 4)
(30, 4)


Agora, iremos treinar os dados utilizando o algoritmo KNN. Dessa forma, o algoritmo KNN foi definido numa classe com os seguintes métodos:

In [6]:
class KNN:
    # Construtor
    def __init__(self, k=3):
        self.k = k

    # Setar o conjunto de treinamento e as classes esperadas
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    # Realizar treinamento
    def predict(self, X):
        y_pred = []
        # Calcular a distância de cada ponto para todos os outros
        for x in X:
            k_idx = self.k_closest_points(x)
            y_pred.append(self.classify(k_idx))

        return np.array(y_pred)

    # Para cada ponto, armazenar em um array a distancia dele para todos os outros
    def k_closest_points(self, x):
        distances = [KNN.euclidian_distance(x, x_train) for x_train in self.X_train]
        # Pegar o k primeiros vizinhos
        k_idx = np.argsort(distances)[: self.k]
        return k_idx

    # Classificação
    def classify(self, k_idx):
        # Extrair os labels dos k primeiros vizinhos
        k_neighbor_labels = [self.y_train[i] for i in k_idx]
        # Obter dos k primeiros vizinhos a classificação, de acordo com a label mais comum
        most_common = Counter(k_neighbor_labels).most_common(1)
        return most_common[0][0]

    # Medida de similaridade (usado a distância euclidiana)
    @staticmethod
    def euclidian_distance(x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    # Acurácia
    @staticmethod
    def accuracy(y_true, y_pred):
        accuracy = np.sum(y_true == y_pred) / len(y_true)
        return accuracy

Agora podemos aplicar os dados ao KNN:

In [7]:
knn = KNN(k=5)
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)
print(predictions)

[1 2 2 0 1 0 0 0 1 2 1 0 2 1 0 1 2 0 2 1 1 1 1 1 2 0 2 1 2 0]


Por fim, iremos testar a acurácia do método, foi criado um método estático na classe KNN para realizar esse cálculo, mas também pode utilizar o método `classification_report` do sklearn:

In [15]:
# Forma 1
print(f"KNN classification accuracy: {knn.accuracy(y_test, predictions)}\n")

# Forma 2
clr = classification_report(y_test, predictions, target_names=iris.target_names)
print(clr)

KNN classification accuracy: 0.9666666666666667

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         9
  versicolor       1.00      0.92      0.96        13
   virginica       0.89      1.00      0.94         8

    accuracy                           0.97        30
   macro avg       0.96      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30

